<a href="https://colab.research.google.com/github/normal-mutant/iron-mapping-tutorial/blob/gh-pages/iron_mapping_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving iron.py to iron.py
Saving Iron_Ore_1.csv to Iron_Ore_1.csv
Saving Iron_Ore_1.xls to Iron_Ore_1.xls
Saving Mapi.html to Mapi.html
Saving world.json to world.json


In [ ]:
data = pd.read_csv("Iron_Ore_1.csv")
data.head()

,METALLOGENESIS,LOCALITY,STATE,TOPOSHEET,LATITUDE,LONGITUDE,LATDD,LONDD,COMMODITY,HOSTROCK,MORPHOGENESIS,FORMATION
0,BABABUDAN RANGE,ATTIGUNDI-GALLIKERA,KARNATAKA,48 O,13 25 N,75 45 E,13.416667,75.750000,Iron,"BHQ,METAPELITE,METAVOLCANICS",VOLCANOSEDIMENTARY-BEDDED,BABABUDAN Sgp.
1,BABABUDAN RANGE,KEMMANAGUNDI,KARNATAKA,48 O,13 32-33 N,75 45 E,13.533333,75.750000,Iron,"BHQ,METAPELITE,METAVOLCANICS",VOLCANOSEDIMENTARY-BEDDED,DHARWAR Sgp.
2,BAILADILA BELT,BAILADILA,CHATTISGARH,65 F,18 40-50 N,81 11-14 E,18.666667,81.183333,Iron,"BHQ,METAPELITE,METAVOLCANICS",VOLCANOSEDIMENTARY-BEDDED,BAILADILA Gp.
3,BELLARY (SANDUR SCHIST BELT),OBULAPURAM,KARNATAKA,57 A,15 04 15 N,76 50 20 E,15.070833,76.838889,Iron,"SHALE,PHYLLITE,BHQ",BEDDED,EASTERN GREENSTONE
4,BONAI-BADAMPAHAR BELT,BADAMGARH,ORISSA,73 G,21 48N,85 16E,21.800000,85.266667,Iron,"BHQ,SHALE INTERLAYERED WITH LAVA",BEDDED-VOLCANOSEDIMENTARY,BONAI Gp.


In [ ]:
#display the datatypes of the columns
data.dtypes

#display coumn names of the data frame
data.columns

#find the unique value of states in the dataframe and  display its counts
print(data['STATE'].unique())
print("--------------------------------")
print(data['STATE'].value_counts())

['KARNATAKA' 'CHATTISGARH' 'ORISSA' 'JHARKHAND' 'TAMILNADU'
 'ANDHRA PRADESH' 'ORISSA ' 'KARNATAKA, A.P' 'MAHARASTRA']
--------------------------------
KARNATAKA         32
ORISSA            24
JHARKHAND          8
TAMILNADU          8
CHATTISGARH        4
ANDHRA PRADESH     1
ORISSA             1
KARNATAKA, A.P     1
MAHARASTRA         1
Name: STATE, dtype: int64


In [ ]:
data.dtypes

METALLOGENESIS     object
LOCALITY           object
STATE              object
TOPOSHEET          object
LATITUDE           object
LONGITUDE          object
LATDD             float64
LONDD             float64
COMMODITY          object
HOSTROCK           object
MORPHOGENESIS      object
FORMATION          object
dtype: object

In [ ]:
#extract the columns of the dataframe  into a python list
lat=list(data["LATDD"])
lon=list(data["LONDD"])
state=list(data["STATE"])
locality=list(data["LOCALITY"])
met=list(data["METALLOGENESIS"])
top=list(data["TOPOSHEET"])
com=list(data["COMMODITY"])

In [ ]:
#create the base layer of map and set it to a location focussing india
map = folium.Map(location=[24.000000, 82.000000], zoom_start=5, tiles="OpenStreetMap")

#map.fit_bounds([[24.000000, 82.000000], [25.000000, 83.000000]])
display(map)

In [ ]:

#function to set different colours of markers to diffent states of india
def color_producer(lii):
    if lii =="JHARKHAND" :
        return "red"
    elif lii=="KARNATAKA":
        return "yellow"
    elif lii=="CHATTISGARH":
        return "blue"
    elif lii=="ORISSA":
        return "green"
    elif lii=="MAHARASTRA":
        return "black"
    elif lii=="TAMILNADU":
        return "purple"
    else:
        return "brown"

In [ ]:
#html that will be show in the popup once a particular marker is clicked
html = """<h4>Iron Ore Resource </h4>
<strong>latitude:</strong>%s <br>
<strong>longitude:</strong>%s <br>
<strong>state:</strong>%s <br>
<strong>locality:</strong>%s <br>
<strong>metallogenisis:</strong>%s <br>
<strong>topsheet:</strong>%s <br>
<strong>commodity:</strong>%s <br>
"""

In [ ]:
#Addding a layer of functionality to the map to show the markers at a particular latitude and longitude location
#creating a new feature group in the map for showing iron ore resources


tile2 = folium.TileLayer('Mapbox Bright')

map.add_child(tile2)

fgi = folium.FeatureGroup(name="Iron")

for lt, ln, state_1,lo,me,to,co in zip(lat, lon,state,locality,met,top,com):

    iframe = folium.IFrame(html=html %(str(lt),str(ln),str(state_1),str(lo),str(me),str(to),str(co)) , width=200,height=300)

    fgi.add_child(folium.CircleMarker(location=[lt, ln], radius = 6, popup=folium.Popup(iframe),

    fill_color=color_producer(state_1), fill=True,  color = 'grey', fill_opacity=0.7))


In [ ]:
#create a new feature group for population
fgp = folium.FeatureGroup(name="Population")

#add population logic
def style_function(x):

    return {'fillColor':'green' if x['properties']['POP2005'] <= 20000000 else 'orange' if 20000000 < x['properties']['POP2005'] < 50000000 else 'red'}

fgp.add_child(folium.GeoJson(data=open('world.json', 'r', encoding='utf-8-sig').read(),

style_function=style_function))

#add both fartures as child(components) to the map

map.add_child(fgp)

map.add_child(fgi)

map.add_child(folium.LayerControl())

#save the map in the form of the html document
map.save("Mapi.html")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# save Mapi.html to google drive
!cp Mapi.html "drive/My Drive/"